# Daten

Funktionen:
- def process_files:  zum Verarbeiten aller Files
- def test_stationarity(data, variable_name): zum Testen der Stationarität eines dataframes -> Ausgabe: Liste mit Tupeln

## Daten importieren und mergen

In [ ]:
#Daten zusammenfügen
import pandas as pd
import glob
import os
import pickle
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.seasonal import STL
from scipy.stats import spearmanr
from scipy.fftpack import fft
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from hampel import hampel
from scipy.stats import skew
from scipy.stats import skewtest


# Verzeichnis mit den Dateien angeben
csv_files_all = "C:/Users/Lotta/OneDrive/Uni/Master/Masterarbeit Python 3.12/Daten/Daten Forward Imputation/*.csv"
dateien = glob.glob(csv_files_all)
    

# Initialisierung der Listen für die verschiedenen Dateitypen
daily_data, hourly_data, weekly_data, monthly_data = [], [], [], []
dfs_hourly, dfs_daily, dfs_weekly, dfs_monthly = [], [], [], []

#Betrachtungszeitraum festlegen
start_date = '2020-12-25' #Daten des Vorjahrs für die Imputation
end_date = '2022-02-01'

# Funktion zum Verarbeiten der Dateien
def process_files(file_list, date_type, dfs_list):
    for file in file_list:
        try:
            # Lese jede CSV-Datei ein
            df = pd.read_csv(file, encoding='ISO-8859-1', sep=';', header=0, dtype={'column': str}, low_memory=False)  # Passe Encoding und Separator an
            df.columns = df.columns.str.replace('ï»¿', '', regex=True)  # Entferne 'ï»¿' beim Datum
            # Überprüfe, ob eine Spalte 'Date' existiert
            if 'Date' in df.columns:
                # Versuche die 'Date'-Spalte in datetime zu konvertieren
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)
                # Filtere die Daten basierend auf dem Betrachtungszeitraum
                df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
                # Lösche alle Spalten, die mit 'Unnamed' anfangen
                df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
                dfs_list.append(df)
            else:
                print(f"Die Datei {file} enthält keine 'Date'-Spalte und wird übersprungen.")
        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file}: {e}")

    # Kombiniere alle DataFrames basierend auf der 'Date'-Spalte
    if dfs_list:
        merged_df = pd.concat(dfs_list, ignore_index=True).sort_values(by='Date')
        merged_df = merged_df.groupby('Date', as_index=False).first()
        return merged_df
    else:
        return pd.DataFrame()

# Zuordnung der Dateien basierend auf ihrem Namen
for file in dateien:
    if 'daily' in os.path.basename(file):
        daily_data.append(file)
    elif 'hourly' in os.path.basename(file):
        hourly_data.append(file)
    elif 'weekly' in os.path.basename(file):
        weekly_data.append(file)
    elif 'monthly' in os.path.basename(file):
        monthly_data.append(file)

# Verarbeite jede Datengruppe und kombiniere die DataFrames
# Verarbeite jede Datengruppe mit Train/Test-Split
df_hourly = process_files(hourly_data, 'hourly', dfs_hourly)
df_daily = process_files(daily_data, 'daily', dfs_daily)
df_weekly = process_files(weekly_data, 'weekly', dfs_weekly)
df_monthly = process_files(monthly_data, 'monthly', dfs_monthly)

# Zusammenfassen der Ergebnisse
merged_dfs = [df_hourly, df_daily, df_weekly, df_monthly]


## Data Preprocessing für imputed data

In [4]:

#Upsampling


#Interpolieren der Daten
for df in merged_dfs: 
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True) # Setze die 'Date'-Spalte als Index
    else:
        print(f"'Date' column not found in dataframe with columns: {df.columns}")

# Resample auf tägliche Daten und Interpolation der numerischen Spalten
daily_df_from_weekly = df_weekly.resample('D').interpolate(method= 'cubic')
daily_from_hourly = df_hourly.resample ('D').median()

 
# Resample die Daten auf täglich und führe eine Interpolation mit cubic durch
for  col in df_monthly:
    if col.startswith('A'):
        df_monthly[col] = df_monthly[col].div(df_monthly.index.days_in_month, axis=0).round(3)
        
daily_from_monthly = df_monthly.resample('D').interpolate(method='cubic')   
# Setze alle Werte bis zum Startdatum der Interpolation auf 0 im daily_from_monthly DataFrame
all_daily_dfs = [daily_df_from_weekly, daily_from_hourly, daily_from_monthly, df_daily]


#combine all daily data frames
fi_final_df = pd.concat(all_daily_dfs, axis=1)
fi_final_df.index = pd.to_datetime(fi_final_df.index)

fi_final_df = fi_final_df.loc[:, ~fi_final_df.columns.str.startswith('Unnamed')]
#Lag von 1 Tag einbaune, da gestrige Daten die neusten Daten sind, die vorliegen
columns_to_lag_one = [col for col in fi_final_df.columns if col != 'S-Gas DA NL']

# Daten um einen Tag verschieben (Lag)
fi_final_df[columns_to_lag_one] = fi_final_df[columns_to_lag_one].shift(1)


# fi_final_df auf drei Nachkommastellen runden
fi_final_df = fi_final_df.round(3)

### Up- und downgesampelte Daten plotten
Optional

In [5]:
"""# Diagramm für wöchentliche Daten
plt.figure(figsize=(15, 6))

# Plot für jede numerische Spalte in daily_df_from_weekly
for column in daily_df_from_weekly.columns:
    plt.plot(daily_df_from_weekly.index, daily_df_from_weekly[column], label=f'Daily Interpolated {column}', linestyle='-', marker='')

# Plot für jede numerische Spalte in merged_df_weekly
for column in merged_df_weekly.columns:
    plt.plot(merged_df_weekly.index, merged_df_weekly[column], label=f'Weekly Data {column}', linestyle='--', marker='o')

# Diagramm beschriften
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Weekly Data vs. Daily Interpolated Data')
plt.legend()
plt.grid(True)

    # Diagramm anzeigen
plt.show()
    # Plot für jede numerische Spalte in daily_df_from_hourly

#Diagramm für stündliche Daten
plt.figure(figsize=(15, 6))
for column in daily_from_hourly.columns:
    plt.plot(daily_from_hourly.index, daily_from_hourly[column], label=f'Daily Interpolated {column}', linestyle='-', marker='')
# Plot für jede numerische Spalte in merged_df_hourly
for column in merged_df_hourly.columns:
    plt.plot(merged_df_hourly.index, merged_df_hourly[column], label=f'Hourly Data {column}', linestyle='--', marker='o')

# Legende, Achsentitel und Anzeige des Plots
plt.xlabel("Datum")
plt.ylabel("Wert")
plt.title("Vergleich: Daily Interpolated vs. Hourly Data")
plt.legend()
plt.grid(True)
plt.show()"""

"""#alle monatlichen in getrennten plots plotten
num_columns = len(daily_from_monthly.columns)  # Anzahl der Spalten
fig, axes = plt.subplots(num_columns, 1, figsize=(15, 6 * num_columns))

# Wenn es nur eine Spalte gibt, sorgt dieser Code dafür, dass axes eine einzelne Achse ist
if num_columns == 1:
    axes = [axes]

# Gehe jede Spalte durch und erstelle ein Diagramm
for i, column in enumerate(daily_from_monthly.columns):
    # Daily Interpolated plot
    axes[i].plot(daily_from_monthly.index, daily_from_monthly[column], label=f'Daily Interpolated {column}', linestyle='-', marker='')

    # Monthly Data plot
    axes[i].plot(merged_df_monthly.index, merged_df_monthly[column], label=f'Monthly Data {column}', linestyle='--', marker='o')

    # Achsenbeschriftungen und Titel
    axes[i].set_xlabel("Datum")
    axes[i].set_ylabel("Wert")
    axes[i].set_title(f"Vergleich: Daily Interpolated vs. Monthly Data ({column})")
    axes[i].legend()
    axes[i].grid(True)

# Layout anpassen, um Überschneidungen zu vermeiden
plt.tight_layout()
plt.show()"""



'#alle monatlichen in getrennten plots plotten\nnum_columns = len(daily_from_monthly.columns)  # Anzahl der Spalten\nfig, axes = plt.subplots(num_columns, 1, figsize=(15, 6 * num_columns))\n\n# Wenn es nur eine Spalte gibt, sorgt dieser Code dafür, dass axes eine einzelne Achse ist\nif num_columns == 1:\n    axes = [axes]\n\n# Gehe jede Spalte durch und erstelle ein Diagramm\nfor i, column in enumerate(daily_from_monthly.columns):\n    # Daily Interpolated plot\n    axes[i].plot(daily_from_monthly.index, daily_from_monthly[column], label=f\'Daily Interpolated {column}\', linestyle=\'-\', marker=\'\')\n\n    # Monthly Data plot\n    axes[i].plot(merged_df_monthly.index, merged_df_monthly[column], label=f\'Monthly Data {column}\', linestyle=\'--\', marker=\'o\')\n\n    # Achsenbeschriftungen und Titel\n    axes[i].set_xlabel("Datum")\n    axes[i].set_ylabel("Wert")\n    axes[i].set_title(f"Vergleich: Daily Interpolated vs. Monthly Data ({column})")\n    axes[i].legend()\n    axes[i].grid

## 1. Data Preprocessing

### 1.1 Data Anomalies - Outliers and Missing values

#### Outliers
- drop all weekend data
- convert UnPlanned Unavailability into binary format
- check whether data frame only hold numerical values
- drop all columns that do not differ from others besides unit
- insert new columns with sum of US export terminals

In [6]:
# Tage extrahieren
fi_final_df['Day'] = fi_final_df.index.weekday+1

fi_final_df = fi_final_df[~fi_final_df['Day'].isin([6,7])]
fi_final_df = fi_final_df.drop(columns=['Day'])

#Check whether the data frame only hold numerical values
for column in fi_final_df:
    if fi_final_df[column].dtype == 'object':
        print(fi_final_df[column].name)
        

#### Handle missing values 
- delete all coumns with missing values > 40%
- interpolate single misisng values linear
- unify units 

In [7]:
## Handling missing values
# Alle missing values behandeln, die nicht interpoliert werden können
fi_final_df = fi_final_df.loc[:, fi_final_df.isnull().mean() < 0.4] #Lösche Spalten mit missing values > 40%
fi_final_df = fi_final_df.interpolate(method='linear')
fi_final_df = fi_final_df.bfill()
#Check whether there are missing values
for col in fi_final_df.columns:
    numberNA = fi_final_df[col].isna().sum()
    if numberNA > 0:
        print(f"Column {col} has {numberNA} missing values.")

## Daten umrechnen & runden
fi_final_df['A-Importiertes Erdgas Deutschland'] = fi_final_df['A-Importiertes Erdgas Deutschland'].mul(0.2778) # Terajoule in GWh
fi_final_df['S-Power price'] = fi_final_df['S-Power price'].mul(10) # ct/KWh in €/MWh
fi_final_df = fi_final_df.round(3)

fi_y_final = fi_final_df[['S-Gas DA NL']]
fi_final_df = fi_final_df.drop(columns=['S-Gas DA NL'])

In [8]:
#1. Feature Selection 
columns_to_drop = [
'N&P-Geschaeftslage (Deutschland)',
    'N&P-Geschaeftserwartungen (Deutschland)',
    'N&P-Geschaeftsklima (verarbeitendes Gewerbe)',
    'N&P-Geschaeftslage (verarbeitendes Gewerbe)',
    'N&P-Geschaeftserwartungen (verarbeitendes Gewerbe)',
    'P-Germany_News_Index',
    'A-Gas in storage (TWh)',
    'A-gas storage Trend (%)',
    'A-Stock/Cons (%)',
    'A-Gas Injection (GWh/d)',
    'N- Gas Withdrawal (GWh/d)',
    'A-Norway Exit Nomination',
    'A-Technical Capacity / DTMI (~GWh)',
    'A-Send-out capacity (GWh/d)',
    'K-Sunshine in s',
    'P-EXY (BidNet)',
    'S-Gas DA D',
    'S-Gas DA UK',
    'S-Gas DA FR',
    'S-Gas DA AU',
    'S-Gas DA IT',
    'S-Coal Channel (Max)',
    'S-Coal Channel (Min)',
    'S-Coal Switching Price Average',
    'A-US LNG Exportterminals',
    'A-Flow Norway to Continent (excl UK)',
    'A-Flow Norway to UK',
    'A-Flow Russian Three Main Lines',
    'A-Flow Russia to Bulgaria (TurkStream 2)',
    'A-Flow Old Russian Routes to Poland (Drozdowicze, Wysokoje, Tietierowka & PWP)',
    'A-Flow Russia to Hungary (Net VIP Bereg)',
    'A-Flow North African Piped',
    'A-Flow UK LNG Sendout',
    'A-Flow Continental LNG Sendout', #wird irgendwie nicht gedropped??
    'A-Flow Azerbaijan via TAP to Italy'
]

# Spalten entfernen
#fi_final_df =fi_final_df.drop(columns=columns_to_drop, errors='ignore')

In [9]:
#Export nach complete set, für Peak
with open('fi_final_df.pkl', 'wb') as f:
    pickle.dump(fi_final_df, f)
with open('fi_y_final.pkl', 'wb') as f:
    pickle.dump(fi_y_final, f)